In [1]:
from torchaudio.models.decoder import ctc_decoder, cuda_ctc_decoder
import numpy as np 
import torch
from torch.nn.functional import softmax, log_softmax
from brainaudio.inference.inference_utils import _cer_and_wer

In [2]:
language_model_path = "/data2/brain2text/lm/languageModel/"
lexicon_phonemes_file = f"{language_model_path}lexicon_phonemes.txt"
units_txt_file_pytorch = f"{language_model_path}units_pytorch.txt"

units_txt_file_pytorch_char = f"{language_model_path}units_pytorch_character.txt"
lexicon_char_file= f"{language_model_path}lexicon_char.txt"

In [3]:
import pandas as pd
val_transcripts = pd.read_pickle("/data2/brain2text/b2t_24/transcripts_val.pkl")

In [6]:
from torchaudio.models.decoder import download_pretrained_files

files = download_pretrained_files("librispeech-4-gram")

In [16]:
decoder_phoneme = ctc_decoder(tokens=units_txt_file_pytorch, lexicon=lexicon_phonemes_file, 
                       beam_size=100, nbest=10, lm=None)
decoder_char = ctc_decoder(tokens=units_txt_file_pytorch_char, lexicon=lexicon_char_file, 
                       beam_size=500, nbest=10, lm=None)

[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached l

In [17]:
model_logits_phone = np.load("/data2/brain2text/b2t_24/logits/tm_transformer_b2t_24_phone/logits_val.npz")
model_logits_char = np.load("/data2/brain2text/b2t_24/logits/tm_transformer_b2t_24_char/logits_val.npz")

In [18]:
ground_truth_arr = []
pred_arr = []
for idx in range(880):
    if idx % 100 == 0:
        print(idx)
    single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)
    beam_search_char = decoder_char(single_trial_logits_char)
    beam_search_transcript_char = " ".join(beam_search_char[0][0].words).strip()
    ground_truth_sentence = val_transcripts[idx]
    ground_truth_arr.append(ground_truth_sentence)
    pred_arr.append(beam_search_transcript_char)
    
cer, wer, wer_sent = _cer_and_wer(pred_arr, ground_truth_arr)
            

0
100
200
300
400
500
600
700
800


In [21]:
pred_arr

['hess recounted',
 'a perse evel und lis goff',
 'so rill we minted in an rash collision',
 'troy cas home need bock loves to be imply alon',
 'the routh vara vote too hom run rauer toth fail out',
 'that singing pare with aus but rad peper i',
 'bear fore thea an am vis e ever forms',
 'reder new suns hase perk',
 "the worden manes it won't bill away oily nothing eased",
 'would a both flasher in a mange hout make him pait o day',
 'he stoke about un atlantic do gangs your',
 'bosh young wray early every moring',
 'with this now loyal gain con creal',
 'mayer rope have an upbeat added',
 "and it's not a inge buy a sis of the pace ord mead the derision",
 'we seed to be rensing the pru lion explosion ret in or on barns',
 'some brines separate measures and all doo in tot moves for the virtue',
 "she lately will be ret owed if if you'll think think im ul erly for fors",
 'she g ake gras frase and prone les enger making cling decision',
 'acme expired kites you delmer',
 'the some xing 

In [20]:
print(wer)

0.35733769776322966


In [13]:

idx = 400
N = 5
T = 3
print("Transcript: ", val_transcripts[idx])


print("\n")
single_trial_logits_phone = torch.as_tensor(model_logits_phone[f'arr_{idx}']).float().unsqueeze(0)
beam_search_phoneme = decoder_phoneme(single_trial_logits_phone/T)
for i in range(N):
    beam_search_transcript_phoneme= " ".join(beam_search_phoneme[0][i].words).strip()
    print(beam_search_transcript_phoneme)

print("\n")
single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)
beam_search_char = decoder_char(single_trial_logits_char/T)
for i in range(N):
    beam_search_transcript_char = " ".join(beam_search_char[0][i].words).strip()
    print(beam_search_transcript_char)
    


Transcript:  friday afternoon at five thirty


void haven't at five laid
void haven't at five lade
void haven't at five lady
void haven't at five thede
void haven't at five thiede


void haven at five play
void havron at five play
void haven at five platy
void havran at five play
void harton at five play


In [117]:
lm_weights = [2.0]   
word_penalty = [0]
sil_score = [-0.1, -0.2, -0.3]
beam_size = [50]

wer_dict = {}
wer_dict['sil'] = []
wer_dict['lmw'] = []
wer_dict['wp'] = []
wer_dict['bs'] = []
wer_dict['wer'] = []


for sil in sil_score:
    for wp in word_penalty:
      for lmw in lm_weights:
          for bs in beam_size:
          
            decoder_char_lm = ctc_decoder(tokens=units_txt_file_pytorch_char, lexicon=lexicon_char_file, 
                                beam_size=bs, nbest=50, lm=files.lm, lm_weight=lmw, word_score=wp, sil_score=sil)

            ground_truth_arr = []
            pred_arr = []
        
            for idx in range(880):
                #if idx % 100 == 0:
                #    print(idx)
                single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)
                beam_search_char = decoder_char_lm(single_trial_logits_char)
                beam_search_transcript_char = " ".join(beam_search_char[0][0].words).strip()
                ground_truth_sentence = val_transcripts[idx]
                ground_truth_arr.append(ground_truth_sentence)
                pred_arr.append(beam_search_transcript_char)
                
            cer, wer, wer_sent = _cer_and_wer(pred_arr, ground_truth_arr)
            
            
            wer_dict['sil'].append(sil)
            wer_dict['lmw'].append(lmw)
            wer_dict['wp'].append(wp)
            wer_dict['bs'].append(bs)
            wer_dict['wer'].append(wer)
            
            
            print(f"sil score: {sil}, lm weight: {lmw}, word penalty: {wp}, beam size: {bs}, wer: {wer}")


sil score: -0.1, lm weight: 2.0, word penalty: 0, beam size: 50, wer: 0.26004728132387706
sil score: -0.2, lm weight: 2.0, word penalty: 0, beam size: 50, wer: 0.259683578832515
sil score: -0.3, lm weight: 2.0, word penalty: 0, beam size: 50, wer: 0.26041098381523914


In [53]:
phoneme_n_best_list = []
for i in range(len(beam_search_phoneme[0])):
    beam_search_transcript_phoneme= " ".join(beam_search_phoneme[0][i].words).strip()
    phoneme_n_best_list.append(beam_search_transcript_phoneme)
    
print(phoneme_n_best_list)

['void after at five flurry', 'void after at five played', 'void after at five plead', 'void after at five platy', 'void after at five purdie', 'void after at five purdy', 'void after at five ploughed', 'void after at five plowed', 'void after at five lady', 'void after at five leedy', 'void after at five leidy', 'void after at fly purdie', 'void after at fly purdy', 'void after at flye purdie', 'void after at flye purdy', 'void after at flye ploughed', 'void after at flye plowed', 'void after at fly plowed', 'void after at fly ploughed', 'void after at five play', 'void after at five plea', 'void e after at five purdy', 'void ee after at five purdie', 'void e. after at five purdy', 'void e after at five purdie', 'void ee after at five purdy', 'void e. after at five purdie', 'void after at fly lady', 'void after at flye lady', 'void e. after at five plowed', 'void e. after at five ploughed', 'void e after at five plowed', 'void ee after at five plowed', 'void ee after at five ploughed'

In [52]:
char_n_best_list = []
for i in range(len(beam_search_char_lm[0])):
    beam_search_transcript_char = " ".join(beam_search_char_lm[0][i].words).strip()
    char_n_best_list.append(beam_search_transcript_char)
    
print(char_n_best_list)

['vida after at five play', 'vide after at five play', 'vida ater at five play', 'vida after at five plays', 'video after at five play', 'vide ater at five play', 'vida after at five player', 'vide after at five plays', 'vida after en at five play', 'vida after at five ploy', 'vida after n at five play', 'vida after at five pla', 'vida after on at five play', 'vida ator at five play', 'vida after at five playa', 'vida after hat five play', 'vide after at five player', 'vida afferent at five play', 'vida ater at five plays', 'vida atra at five play', 'video ater at five play', 'vida aren at five play', 'vida arn at five play', 'vida datron at five play', 'veda after at five play', 'vide after en at five play', 'vide after at five ploy', 'vide after n at five play', 'video after at five plays', 'vida aron at five play', 'vida tern at five play', 'vida after at five flay', 'vida fern at five play', 'vida aten at five play', 'vide after at five pla', 'vide after on at five play', 'vida ate

In [101]:

single_trial_logits_char = torch.as_tensor(model_logits_char[f'arr_{idx}']).float().unsqueeze(0)


In [56]:
start = time.time()

end = time.time()
print(end-start)

0.013104915618896484


In [16]:
single_trial_logits.shape

torch.Size([1, 61, 41])

In [15]:
decoder.decode_begin()
for t in range(32):
    decoder.decode_step(single_trial_logits[0, t:t + 1, :])
    
decoder.decode_end()
beam_search_result_inc = decoder.get_final_hypothesis()
print(beam_search_result_inc)

[CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40,  2, 14, 31, 12, 40]), words=['VOID', 'AFTER'], score=522.8263626098633, timesteps=tensor([ 0, 10, 12, 14, 17, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40, 18, 40,  2, 14, 31, 12, 40]), words=['VOID', 'E.', 'AFTER'], score=520.3375873565674, timesteps=tensor([ 0, 10, 12, 14, 16, 17, 18, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40, 18, 40,  2, 14, 31, 12, 40]), words=['VOID', 'E', 'AFTER'], score=520.3375873565674, timesteps=tensor([ 0, 10, 12, 14, 16, 17, 18, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40, 18, 40,  2, 14, 31, 12, 40]), words=['VOID', 'EE', 'AFTER'], score=520.3375873565674, timesteps=tensor([ 0, 10, 12, 14, 16, 17, 18, 22, 24, 25, 27, 31], dtype=torch.int32)), CTCHypothesis(tokens=tensor([40, 35, 26,  9, 40,  2, 14, 31, 12, 23, 34, 23, 40]), words=['VOID', 'AFTERNOON'], score=519.06853389

In [59]:
import heapq

def make_new_beam():
  fn = lambda : (NEG_INF, NEG_INF)
  return collections.defaultdict(fn)

def logsumexp(*args):
  """
  Stable log sum exp.
  (Optimized version)
  """
  if not args:
      return NEG_INF

  # <-- 2. Optimized logsumexp
  # Find max in one pass
  a_max = args[0]
  for a in args[1:]:
      if a > a_max:
          a_max = a

  if a_max == NEG_INF:
      return NEG_INF

  # Calculate sum in a second pass
  lsp_sum = 0.0
  for a in args:
      lsp_sum += math.exp(a - a_max)
      
  return a_max + math.log(lsp_sum)

def decode(probs, beam_size=100, blank=0):
  """
  Performs inference for the given output probabilities.
  Arguments:
      probs: The output probabilities (e.g. post-softmax) for each
        time step. Should be an array of shape (time x output dim).
      beam_size (int): Size of the beam to use during inference.
      blank (int): Index of the CTC blank label.
  Returns the output label sequence and the corresponding negative
  log-likelihood estimated by the decoder.
  """
  T, S = probs.shape
  # Convert to log-probabilities once at the beginning
  log_probs = np.log(probs)

  # Elements in the beam are (prefix, (p_blank, p_no_blank))
  # Initialize the beam with the empty sequence, a probability of
  # 1 for ending in blank and zero for ending in non-blank
  # (in log space).
  beam = [(tuple(), (0.0, NEG_INF))]

  for t in range(T): # Loop over time

    # A default dictionary to store the next step candidates.
    next_beam = make_new_beam()

    # <-- 3. Pre-slice probabilities for this time step
    log_probs_t = log_probs[t, :]

    for s in range(S): # Loop over vocab
      p = log_probs_t[s] # Use the pre-sliced array

      # The variables p_b and p_nb are respectively the
      # probabilities for the prefix given that it ends in a
      # blank and does not end in a blank at this time step.
      for prefix, (p_b, p_nb) in beam: # Loop over beam

        # If we propose a blank the prefix doesn't change.
        # Only the probability of ending in blank gets updated.
        if s == blank:
          n_p_b, n_p_nb = next_beam[prefix]
          n_p_b = logsumexp(n_p_b, p_b + p, p_nb + p)
          next_beam[prefix] = (n_p_b, n_p_nb)
          continue

        # Extend the prefix by the new character s and add it to
        # the beam. Only the probability of not ending in blank
        # gets updated.
        end_t = prefix[-1] if prefix else None
        n_prefix = prefix + (s,)
        n_p_b, n_p_nb = next_beam[n_prefix]
        if s != end_t:
          n_p_nb = logsumexp(n_p_nb, p_b + p, p_nb + p)
        else:
          # We don't include the previous probability of not ending
          # in blank (p_nb) if s is repeated at the end. The CTC
          # algorithm merges characters not separated by a blank.
          n_p_nb = logsumexp(n_p_nb, p_b + p)
          
        next_beam[n_prefix] = (n_p_b, n_p_nb)

        # If s is repeated at the end we also update the unchanged
        # prefix. This is the merging case.
        if s == end_t:
          n_p_b, n_p_nb = next_beam[prefix]
          n_p_nb = logsumexp(n_p_nb, p_nb + p)
          next_beam[prefix] = (n_p_b, n_p_nb)

    # <-- 4. MAJOR OPTIMIZATION: Use heapq.nlargest instead of full sort
    # Original:
    # beam = sorted(next_beam.items(),
    #         key=lambda x : logsumexp(*x[1]),
    #         reverse=True)
    # beam = beam[:beam_size]
    
    # Optimized:
    beam_items = next_beam.items()
    beam = heapq.nlargest(beam_size,
                          beam_items,
                          key=lambda x: logsumexp(*x[1]))

  # Final selection
  best = beam[0]
  return best[0], -logsumexp(*best[1])

if __name__ == "__main__":
  np.random.seed(3)

  time = 50
  output_dim = 20

  probs = np.random.rand(time, output_dim)
  probs = probs / np.sum(probs, axis=1, keepdims=True)

  labels, score = decode(probs)
  print(labels)
  print("Score {:.3f}".format(score))

(5, 12, 8, 16, 5, 13, 19, 7, 19, 3, 2, 4, 11, 19, 8, 14, 18, 2, 13, 17, 14, 18, 5, 1, 6, 9)
Score 97.032


In [63]:
from scipy.special import softmax
logits = softmax(model_logits[f'arr_750}'],axis=1)
import time

In [65]:
start = time.time()
labels, probs = decode(logits)
end = time.time()
print(end-start)

0.6410787105560303
